租房信息
区域，小区，地址，租金，户型，出租方式，特色，朝向，面积，楼层，装修，房源亮点，周边配套，交通出行，配套设施，经纪人

楼盘信息
index,url,house,alias,price,price_detail,room_type,direct,decoration,onsaledate,completedate,surroundingfacilities,commentrank,salesphone,projectmap


In [1]:
import requests
import re
from pyquery import PyQuery as pq
import random

In [2]:
index = 1
url = 'http://xiangjiangtianfu.fang.com/'
url_detail='http://xiangjiangtianfu.fang.com/house/2811174364/housedetail.htm'

In [20]:
user_agent_list = [
"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
"Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
"Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
"Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
"Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
"Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
"Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
"Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
"Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
]

headers = {
    'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Host':'xiangjiangtianfu.fang.com',
    'Referer':'http://gz.newhouse.fang.com/house/s/',
    'Upgrade-Insecure-Requests':'1',
    'User-Agent': random.choice(user_agent_list)
}

In [42]:
rq = requests.get(url,headers=headers)
#gzip解码
doc=pq(rq.content.decode('gb18030'))

In [ ]:
# rq.encoding = 'latin-1'
# rq.content

In [44]:
#被gzip压缩必须先解码
doc('body > div.main_1200.tf > div.firstbox > div.firstright.fr > div.information > div.inf_left1 > div > h1 > strong').text()

'香江天赋'

In [7]:
#另一种解码方法
import gzip
import urllib
req = urllib.request.Request(url=url,headers=headers,method='GET')
content_ori = urllib.request.urlopen(req).read()
content = gzip.decompress(content_ori).decode("gb18030")

In [45]:
#另一种解码方法
rq.text.encode('latin-1').decode("gb18030")

'<!doctype html>\n<html>\n<head>\n<title>\n\t香江天赋怎么样_房价_周边配套_地址_售楼电话 - 广州房天下\n</title>\n<meta name="Keywords" content="香江天赋, 广州香江天赋，香江天赋楼盘详情" />\n<meta name="Description" content="广州香江天赋，12月房价(参考均价：23900元/平方米)，售楼电话(400-890-0000 转 885462)，位于增城商圈，楼盘地址是广园快速路开创大道，占地面积96784平方米，拥有70年产权，由&#160;增城香江房地产有限公司负责开发"/>\n<meta http-equiv="Content-Type" content="text/html; charset=gb2312">\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />\n<meta name="mobile-agent" content="format=html5;url=https://m.fang.com/xf/gz/2811174364.htm">\n<link rel="alternate" media="only screen and(max-width: 640px)" href="https://m.fang.com/xf/gz/2811174364.htm" >\n<meta name="applicable-device" content="pc">\n<meta http-equiv="Content-Language" content="zh-cn">\n<link rel="alternate" media="only screen and(max-width: 640px)" href="https://m.fang.com/xf/gz/2811174364.htm " >\n\n<script type="text/javascript" src="//static.soufunimg.com/zbs/newhouse/js/jquery-1.7.2.min.js"></script>\n\n\n\n\n\n\n<!-- 201611

In [46]:
doc=pq(content)

In [47]:
house=doc('body > div.main_1200.tf > div.firstbox > div.firstright.fr > div.information > div.inf_left1 > div > h1 > strong').text()
alias=doc('body > div.main_1200.tf > div.firstbox > div.firstright.fr > div.information > div.inf_left1 > div > span').text()
price=doc('body > div.main_1200.tf > div.firstbox > div.firstright.fr > div.information > div:nth-child(2) > div.inf_left.fl > span').text()
price_detail=doc('body > div.main_1200.tf > div.firstbox > div.firstright.fr > div.information > div:nth-child(2) > div.inf_right.fl.pr > a.pra01.fl > div > p:nth-child(2)').text()
print(house)
print(alias)
print(price)
print(price_detail)


香江天赋
别名：香江翡翠馨园
23900
香江天赋在售复式及平层单位，88㎡三房平层单位均价23000元/平，98-127㎡四房或五房复式单位均价24800元/平，均为带装修出售。


In [5]:
from lxml import etree
et=etree.HTML(content)

In [52]:
et.xpath('/html/body/div[3]/div[3]/div[2]/div[1]/div[2]/div[2]/a[1]/div/p[1]/text()')

['\n\t\t\t\t\t\t香江天赋在售复式及平层单位，88㎡三房平层单位均价23000元/平，98-127㎡四房或五房复式单位均价24800元/平，均为带装修出售。 \n\t\t\t\t\t\t\t\t\t\t\t\t']

In [7]:
"""
req_d = urllib.request.Request(url=url_detail,headers=headers,method='GET')
content_d = urllib.request.urlopen(req_d).read()
content_d = gzip.decompress(content_d).decode("gb18030")
doc_d=pq(content_d)
"""

In [56]:
req_d.content.decode("gb2312")

UnicodeDecodeError: 'gb2312' codec can't decode byte 0x8b in position 45456: illegal multibyte sequence

In [52]:
req_d = requests.get(url_detail,headers=headers)
doc_d = pq(req_d.content.decode("gb18030"))

In [55]:
#基本信息
basic_info=doc_d('body > div:nth-child(46) > div > div.main-left > div:nth-child(1)')
room_type=doc_d('body > div:nth-child(46) > div > div.main-left > div:nth-child(1) > div > div.main-info-price > em')
# direct=
# decoration
# onsaledate=
# completedate

# surroundingfacilities
# commentrank
# salesphone
# projectmap
print(basic_info)
print(room_type)
print(doc_d)



<html xmlns="http://www.w3.org/1999/xhtml" xmlns="http://www.w3.org/1999/xhtml">
	<head>
		<meta http-equiv="Content-Type" content="text/html; charset=gb2312" />
		<title>香江天赋基本信息,售楼处电话,物业费,物业电话,开发商-广州房天下</title>
		<meta name="keywords" content="香江天赋售楼处地址,香江天赋售楼电话,香江天赋物业费电话,香江天赋开发商" />
		<meta name="description" content="广州房天下为您提供香江天赋基本信息，包括售楼处地址，售楼电话，物业费，物业电话，开发商等香江天赋楼盘相关信息，为您选房、买房提供参考!" />
		<meta name="mobile-agent" content="format=html5;url=https://m.fang.com/xf/gz/2811174364.htm" />
		<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />
		<script src="//static.soufunimg.com/common_m/pc_public/fangjs/build/ie8/ie8.3.js" type="text/javascript"></script>
        <link href="//static.soufunimg.com/newhouse/style/css2_bj.css?v=11" rel="stylesheet" type="text/css" />
		<link href="//static.soufunimg.com/zbs/newhouse/css/sfpc_base.css" rel="stylesheet" type="text/css" />
		<link href="//static.soufunimg.com/newhouse/style/detailcss_201608.css?v=20181025" type="text/css" rel

https://www.jianshu.com/p/639a67bc75b9

In [48]:
basic_info=doc_d('body').text()
basic_info

'var xf_adshow_ctrl = \'xfdetail_ad_ctrl\'; function loadScript(B,D){var A=document.createElement("script"),C=document.documentElement.firstChild;A.type="text/javascript";if(A.readyState){A.onreadystatechange=function(){if(A.readyState=="loaded"||A.readyState=="complete"){A.onreadystatechange=null;D()}}}else{A.onload=function(){D()}}A.src=B;C.insertBefore(A,C.firstChild)}; var city = "广州";\n香江天赋\n400-890-0000\xa0转\xa0885462\n扫描下载\n房天下APP\nvar sign = \'ds\'; var newhouseDomain = \'//gz.newhouse.fang.com\'; var newcode = \'2811174364\'; var adDmOpen = \'1\'; var jiaju_ad = \'\'; var sem = \'\'; var jiaju_point = 0; var barrage; var ajaxUrl = "/house/ajaxrequest/adXml_get.php";\n广州\n热门城市\nABCDFGH\nJKLMNQST\nWXYZ\n全国 北京 上海 广州 深圳 成都 重庆 天津 武汉 杭州 南京 苏州 济南 石家庄 西安 郑州 无锡 青岛 南昌 东莞 大连 昆明 海南 三亚 香港 海外 更多\n全国 北京 上海 广州 深圳 成都 重庆 天津 武汉 杭州 南京 苏州 济南 石家庄 西安 郑州 无锡 青岛 南昌 东莞 大连 昆明 海南 三亚\n济南 江门 昆山 昆明 廊坊 洛阳 兰州 南京 宁波 南通 南昌 南宁 青岛 秦皇岛 沈阳 石家庄 上海 深圳 苏州 绍兴 三亚 天津 唐山 太原\n武汉 无锡 潍坊 威海 西安 徐州 厦门 烟台 扬州 银川 郑州 舟山 珠海\n首页\n新房\n

In [29]:
x=u"\u8def\u52b2\u5929\u96bd\u5cf0\u03b626000\u03b6\u5e7f\u5dde\u03b6"
x

'路劲天隽峰ζ26000ζ广州ζ'

In [27]:
x={"status":"succ","desc":"OK","data":[{"keyname":"2811174364","value":"\u9999\u6c5f\u5929\u8d4b\u03b623900\u03b6\u5e7f\u5dde\u03b6http:\/\/xiangjiangtianfu.fang.com\/","time":"1551572540"},{"keyname":"2811172238","value":"\u8def\u52b2\u5929\u96bd\u5cf0\u03b626000\u03b6\u5e7f\u5dde\u03b6http:\/\/tianjuanfenglj.fang.com\/","time":"1551541876"},{"keyname":"2811173782","value":"\u9996\u521b\u78a7\u6842\u56ed\u60a6\u5c71\u5e9c\u03b621500\u03b6\u5e7f\u5dde\u03b6http:\/\/yueshanfuscbgy.fang.com\/","time":"1551537514"},{"keyname":"2811174354","value":"\u57ce\u6295\u00b7\u9996\u7b51\u82b1\u56ed\u03b619500\u03b6\u5e7f\u5dde\u03b6http:\/\/shouzhuhuayuanct.fang.com\/","time":"1551533911"},{"keyname":"2811174346","value":"\u5408\u5bb6\u548c\u5e9c\u03b615500\u03b6\u5e7f\u5dde\u03b6http:\/\/hejiahefu.fang.com\/","time":"1551533909"},{"keyname":"2811173600","value":"\u4e07\u79d1A32\u03b618500\u03b6\u5e7f\u5dde\u03b6http:\/\/173600.fang.com\/","time":"1551533902"},{"keyname":"2811174394","value":"\u5408\u6c47\u00b7\u749f\u56ed\uff08\u5546\u7528\uff09\u03b6\u5f85\u5b9a\u03b6\u5e7f\u5dde\u03b6http:\/\/hehuijingyuan020.fang.com\/","time":"1551533832"},{"keyname":"2811174398","value":"\u8fdc\u8fbe\u5e7f\u573a\u03b6\u5f85\u5b9a\u03b6\u5e7f\u5dde\u03b6http:\/\/yuandaguangchang020.fang.com\/","time":"1551533830"},{"keyname":"2811184668","value":"\u81ea\u7531\u4eba\u82b1\u56ed\u03b624000\u03b6\u5e7f\u5dde\u03b6http:\/\/ziyourenhuayuan.fang.com\/","time":"1551533779"}]}


api:https://ditu.fang.com/?c=channel&a=ajaxXiaoquMapSearch&x1=113.557556&y1=23.1417&distance=2&strNewCode=2811174364&city=gz